In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.linear_model
from pathlib import Path

In [6]:
path_ds:Path = Path(".") / "datasets"

In [7]:
#Cargar la data
df = pd.read_csv(path_ds / 'DEvideos.csv')
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,state,lat,lon,geometry
0,LgVi6y5QIjM,17.14.11,Sing zu Ende! | Gesangseinlagen vom Feinsten |...,inscope21,24,2017-11-13T17:08:49.000Z,"inscope21|""sing zu ende""|""gesangseinlagen""|""ge...",252786,35885,230,1539,https://i.ytimg.com/vi/LgVi6y5QIjM/default.jpg,False,False,False,Heute gibt es mal wieder ein neues Format... w...,Schleswig Holstein,54.783748,9.433315,POINT (9.433315388 54.78374778)
1,Bayt7uQith4,17.14.11,Kinder ferngesteuert im Kiosk! Erwachsene abzo...,LUKE! Die Woche und ich,23,2017-11-12T22:30:01.000Z,"Kinder|""ferngesteuert""|""Kinder ferngesteuert""|...",797196,53576,302,1278,https://i.ytimg.com/vi/Bayt7uQith4/default.jpg,False,False,False,Kinder ferngesteuert! Kinder lassen sich sooo ...,Nordrhein Westfalen,52.029988,8.530011,POINT (8.530011351000001 52.02998822)
2,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97190,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",Saarland,49.250390,6.970003,POINT (6.970003213 49.25039044)
3,AHtypnRk7JE,17.14.11,Das Fermi-Paradoxon,100SekundenPhysik,27,2017-11-12T15:00:01.000Z,"Physik|""Wissenschaft""|""Technik""|""Science-Ficti...",380247,31821,458,1955,https://i.ytimg.com/vi/AHtypnRk7JE/default.jpg,False,False,False,►Alle Videos: http://bit.ly/1fa7Tw3\n\n\n✚Snap...,Bayern,50.266607,10.966607,POINT (10.96660681 50.26660748)
4,ZJ9We4bjcg0,17.14.11,18 SONGS mit Kelly MissesVlog (Sing-off),rezo,24,2017-11-12T13:10:36.000Z,"kelly|""missesvlog""|""kelly song""|""bausa""|""bausa...",822213,100684,2467,10244,https://i.ytimg.com/vi/ZJ9We4bjcg0/default.jpg,False,False,False,18 Song Mashup über den (veränderten) Beat von...,Berlin,52.521819,13.401549,POINT (13.40154862 52.52181866)


In [16]:
df.isnull().sum()

video_id                     0
trending_date                0
title                        0
channel_title                0
category_id                  0
publish_time                 0
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
thumbnail_link               0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
description               1552
state                        0
lat                          0
lon                          0
geometry                     0
dtype: int64

Al ver los valores nulos nos damos cuenta que hay `1552` valores vacios en la columna description, esto en principio no perjdicará el analisis y el modelo ya que es una variable cualitativa y en este contexto muchas veces los videos de YouTube se suben sin descripción y no es algo que se tome muy en cuenta al mirar un video

In [17]:
# obtener columnas importantes
columnas_importantes = ["video_id", "trending_date", "title", "channel_title",
                        "category_id", "publish_time", "tags", "views", 
                        "likes", "dislikes", "comment_count"
                    ]

In [20]:
df_filter = df[columnas_importantes]
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40840 entries, 0 to 40839
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   video_id       40840 non-null  object
 1   trending_date  40840 non-null  object
 2   title          40840 non-null  object
 3   channel_title  40840 non-null  object
 4   category_id    40840 non-null  int64 
 5   publish_time   40840 non-null  object
 6   tags           40840 non-null  object
 7   views          40840 non-null  int64 
 8   likes          40840 non-null  int64 
 9   dislikes       40840 non-null  int64 
 10  comment_count  40840 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 3.4+ MB


In [22]:
len(df_filter.video_id.unique())

29627

El `video_id` es el identificador de cada video, al sacar los valores únicos vemos que hay muchos menos que en el dataframe original, por lo tanto se puede decir que hay valores repetidos, hay que ver cuales son esos valores para saber que hacer con ellos.

In [24]:
primer_id = df_filter.video_id.iloc[0]
df_filter[df_filter.video_id == primer_id]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count
0,LgVi6y5QIjM,17.14.11,Sing zu Ende! | Gesangseinlagen vom Feinsten |...,inscope21,24,2017-11-13T17:08:49.000Z,"inscope21|""sing zu ende""|""gesangseinlagen""|""ge...",252786,35885,230,1539
203,LgVi6y5QIjM,17.15.11,Sing zu Ende! | Gesangseinlagen vom Feinsten |...,inscope21,24,2017-11-13T17:08:49.000Z,"inscope21|""sing zu ende""|""gesangseinlagen""|""ge...",551806,59963,875,2344
423,LgVi6y5QIjM,17.16.11,Sing zu Ende! | Gesangseinlagen vom Feinsten |...,inscope21,24,2017-11-13T17:08:49.000Z,"inscope21|""sing zu ende""|""gesangseinlagen""|""ge...",632932,65572,1022,2554
678,LgVi6y5QIjM,17.17.11,Sing zu Ende! | Gesangseinlagen vom Feinsten |...,inscope21,24,2017-11-13T17:08:49.000Z,"inscope21|""sing zu ende""|""gesangseinlagen""|""ge...",673401,68398,1068,2590


Esta salida nos muestra el progreso del video, como es que le ha ido en sus dias de trending, este primer video estuvo en trending por 4 días, desde el 14 de noviembre de 2017 hasta el 17 de noviembre del 2017, para cada uno de los días, tenemos su informació, como el numero de vistas, los likes, dislikes y cantidad de comentarios